In [3]:

from pymongo import MongoClient
conn = MongoClient("localhost:27017")
import pandas as pd
from pandas import json_normalize
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv

load_dotenv()

In [4]:
conn.list_database_names()
db = conn.get_database("ironhack")
db.list_collection_names()
collection = db.get_collection("companies")

In [5]:
#Expoloro lo que hay en Companies para pensar un estrategia para busca la ubicación de mi empresa

In [6]:
documents2 = collection.find( {'number_of_employees': {'$lte': 10}})

In [7]:
data_aux = pd.DataFrame(list(documents2))
data_aux.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0,...,"[{'funding_round': {'round_code': 'b', 'source...",None,"[{'price_amount': 20000000, 'price_currency_co...","[{'description': '', 'address1': '407 N Maple ...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 94], 'assets/imag...",[],[]
1,52cdef7c4bab8bd675297d9d,Joost,joost,http://www.crunchbase.com/company/joost,http://joost.com,http://blog.joost.com,http://blog.joost.com/atom.xml,None,games_video,0,...,[],"{'price_amount': None, 'price_currency_code': ...",[],"[{'description': '', 'address1': '100 5th Ave ...","[{'id': 4860, 'description': 'Joost Video Netw...",None,"[{'embed_code': '<object width=""425"" height=""3...",[],[],[]
2,52cdef7c4bab8bd675297dbc,Skype,skype,http://www.crunchbase.com/company/skype,http://www.skype.com,http://blogs.skype.com,http://feeds.feedburner.com/shareskypeen,skype,software,0,...,"[{'funding_round': {'round_code': 'a', 'source...","{'price_amount': 2600000000, 'price_currency_c...","[{'price_amount': 150000000, 'price_currency_c...","[{'description': 'Global HQ', 'address1': '23-...","[{'id': 552, 'description': 'Skype released pu...",None,"[{'embed_code': '<iframe src=""http://blip.tv/p...","[{'available_sizes': [[[150, 130], 'assets/ima...",[{'external_url': 'http://www.ewdn.com/2011/06...,[]
3,52cdef7c4bab8bd675297dbd,YouTube,youtube,http://www.crunchbase.com/company/youtube,http://www.youtube.com,http://www.youtube.com/blog,http://youtube-global.blogspot.com/feeds/posts...,YouTube,games_video,0,...,[],"{'price_amount': 1650000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 989, 'description': 'YouTube Hits 100 ...",None,"[{'embed_code': '<iframe width=""430"" height=""2...","[{'available_sizes': [[[150, 125], 'assets/ima...",[],[]
4,52cdef7c4bab8bd675297dc5,GrandCentral,grandcentral,http://www.crunchbase.com/company/grandcentral,http://grandcentral.com,http://blog.grandcentral.com/,http://blog.grandcentral.com/?feed=rss2,,mobile,3,...,[],"{'price_amount': 60000000, 'price_currency_cod...",[],"[{'description': None, 'address1': '48389 Frem...",[],None,"[{'embed_code': '<object width=""425"" height=""3...",[],[],[]


In [8]:
data_aux.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'ipo', 'video_embeds',
       'screenshots', 'external_links', 'partners'],
      dtype='object')

In [9]:
data_aux['category_code'].unique()

array(['web', 'games_video', 'software', 'mobile', 'enterprise', 'search',
       'network_hosting', 'ecommerce', 'photo_video', 'advertising',
       'social', 'finance', 'news', 'analytics', 'travel',
       'public_relations', 'education', 'security', 'consulting', 'music',
       'cleantech', 'legal', 'messaging', 'hardware', 'other',
       'hospitality', None, 'real_estate', 'biotech', 'sports', 'health',
       'medical', 'nonprofit', 'design', 'fashion'], dtype=object)

In [10]:
# Creo una búsqueda basada en los siguientes criterios:
# 1- Selecciono todas las empresas tecnológicas.
# 2- Tienen que tener al menos un empleado.
# 3- Selecciono los documentos en los que el valor del campo no es nulo
# 4- Selecciono las startups filtradon por el año de su fundación (después 2008) y por le número de empleados (<20)

In [11]:
companies = collection.find(
    {'$and':[
        {'$or':[
            {'category_code': 'web'},
            {'category_code': 'games_video'},
            {'category_code':'software'},
            {'category_code':'mobile'},
            {'category_code':'network_hosting'},
            {'category_code':'ecommerce'},
            {'category_code':'analytics'},
            {'category_code':'consulting'},
            {'category_code':'cleantech'},
            {'category_code':'biotech'},
            {'category_code':'design'}]},
        {'number_of_employees': {'$lte': 20}},
        {'number_of_employees': {'$ne': None}},
        {'number_of_employees': {'$ne': 0}},
        {'offices.latitude': {'$ne': None}},
        {'offices.longitude': {'$ne': None}},
        {'offices.city': {'$ne': None}},
        {'offices.country_code': {'$ne': None}},
        {'name': {'$ne': None}},
        {'founded_year': {'$gte': 2008}}, 
        {'offices': { '$ne': []} }
    ]},
    {'name' : 1, 'category_code' : 1, 'number_of_employees' : 1,
     'founded_year' : 1, 'offices.country_code': 1, 'offices.city': 1, 'offices.latitude' : 1, 'offices.longitude' : 1, '_id': 0})



In [12]:
# Convierto los datos a dataframe y empiezo la limpieza

In [13]:
data = json_normalize(data = companies, record_path = 'offices', meta = ['name', 'category_code', 'number_of_employees', 'founded_year'])

In [14]:
data.head()

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
0,Bellevue,USA,47.597965,-122.151158,YouBeQB,games_video,8,2008
1,Norderstedt,DEU,53.707739,10.023246,alluc,games_video,7,2009
2,Santa Monica,USA,34.031764,-118.461884,WonderHowTo,games_video,12,2008
3,Louisville,USA,38.257035,-85.717393,First30Days,web,17,2008
4,Kirkland,USA,47.704875,-122.187925,PokerPlayersSocialNetwork,web,3,2008


In [15]:
data.size

6864

In [16]:
#Selecciono las empresas de Estados Unidos

In [17]:
data_esp = data.loc[data['city'] == "Madrid"] 

In [18]:
data_esp

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
224,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008
292,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008
555,Madrid,ESP,40.607090,-3.691275,Zankyou,web,15,2008
773,Madrid,ESP,40.636710,-3.998048,Undead Code Studios,games_video,5,2009
821,Madrid,ESP,40.424459,-3.699095,Taxi Fares app,software,5,2009


In [19]:
def get_geo(a, b):
    return {'type': 'Point', 'coordinates': [a, b]}

In [20]:
data_esp['geopoint'] = data_esp.apply(lambda x: get_geo(x['longitude'], x['latitude']), axis = 1)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
data_esp.head(10)

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year,geopoint
224,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008,"{'type': 'Point', 'coordinates': [-3.703248, 4..."
292,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008,"{'type': 'Point', 'coordinates': [-3.703248, 4..."
555,Madrid,ESP,40.607090,-3.691275,Zankyou,web,15,2008,"{'type': 'Point', 'coordinates': [-3.6912754, ..."
773,Madrid,ESP,40.636710,-3.998048,Undead Code Studios,games_video,5,2009,"{'type': 'Point', 'coordinates': [-3.9980485, ..."
821,Madrid,ESP,40.424459,-3.699095,Taxi Fares app,software,5,2009,"{'type': 'Point', 'coordinates': [-3.6990948, ..."


In [22]:
data_esp.dropna( inplace=True )


Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
data_esp.head(10)

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year,geopoint
224,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008,"{'type': 'Point', 'coordinates': [-3.703248, 4..."
292,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008,"{'type': 'Point', 'coordinates': [-3.703248, 4..."
555,Madrid,ESP,40.607090,-3.691275,Zankyou,web,15,2008,"{'type': 'Point', 'coordinates': [-3.6912754, ..."
773,Madrid,ESP,40.636710,-3.998048,Undead Code Studios,games_video,5,2009,"{'type': 'Point', 'coordinates': [-3.9980485, ..."
821,Madrid,ESP,40.424459,-3.699095,Taxi Fares app,software,5,2009,"{'type': 'Point', 'coordinates': [-3.6990948, ..."


In [24]:
data.to_json('cleaned_data_esp.json', orient = 'records', lines = True)

In [25]:
#Obtengo los datos de la api

In [26]:
client_id = os.getenv("CLIENT_ID")
token = os.getenv("CLIENT_SECRET")
import json
import requests


In [27]:
def get_place_coords(lat, long, place, typeC):
    try:
        url = 'https://api.foursquare.com/v2/venues/explore'
        parametros = { "client_id" : client_id,
              "client_secret" : token,
              "v" : "20180323",
              "ll": f"{lat},{long}",
              "query": place,
              "limit" : 1
        }
        resp = requests.get(url=url, params=parametros)
        data = json.loads(resp.text)
        lat = data['response']['groups'][0]['items'][0]['venue']['location']['lat']
        long = data['response']['groups'][0]["items"][0]["venue"]["location"]["lng"]
        return long if typeC == "long" else lat
    except IndexError:
        return None

In [28]:
def geta():
    for a in gdfStarbucks['geometry']:
        return a
    

In [29]:
data_esp['lat_vegan'] = data_esp.apply(lambda x: get_place_coords(x.latitude, x.longitude, "vegan", "lat"), axis=1)
data_esp['long_vegan'] = data_esp.apply(lambda x: get_place_coords(x.latitude, x.longitude, "vegan", "long"), axis=1)



C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
data_esp['lat_airport'] = data_esp.apply(lambda x: get_place_coords(x.latitude, x.longitude, "airport", "lat"), axis=1)
data_esp['long_airport'] = data_esp.apply(lambda x: get_place_coords(x.latitude, x.longitude, "airport", "long"), axis=1)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
data_esp

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year,geopoint,lat_vegan,long_vegan,lat_airport,long_airport
224,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008,"{'type': 'Point', 'coordinates': [-3.703248, 4...",40.410672,-3.705982,40.481654,-3.578024
292,Madrid,ESP,40.412323,-3.703248,Daily Flat Rental,ecommerce,2,2008,"{'type': 'Point', 'coordinates': [-3.703248, 4...",40.410672,-3.705982,40.481654,-3.578024
555,Madrid,ESP,40.607090,-3.691275,Zankyou,web,15,2008,"{'type': 'Point', 'coordinates': [-3.6912754, ...",40.531880,-3.644637,40.481654,-3.578024
773,Madrid,ESP,40.636710,-3.998048,Undead Code Studios,games_video,5,2009,"{'type': 'Point', 'coordinates': [-3.9980485, ...",40.451159,-3.704319,40.491670,-3.969001
821,Madrid,ESP,40.424459,-3.699095,Taxi Fares app,software,5,2009,"{'type': 'Point', 'coordinates': [-3.6990948, ...",40.423475,-3.700037,40.481654,-3.578024


In [ ]:
#Visualizo los datos

In [32]:
inicial_lat = 40.4143851
inicial_long = -3.6820241
map_1 = folium.Map(location = [inicial_lat,inicial_long], zoom_start =11)
companies_group = folium.FeatureGroup(name = "Companies")
HeatMap(data = data_esp[["latitude","longitude"]], radius = 15).add_to(companies_group)
companies_group.add_to(map_1)
map_1

In [33]:
airports_group = folium.FeatureGroup (name = "Airports")
HeatMap(data = data_esp[["lat_airport","long_airport"]], radius = 15).add_to(airports_group)
airports_group.add_to(map_1)
map_1

In [34]:
vegan_group = folium.FeatureGroup (name = "Vegans")
HeatMap(data = data_esp[["lat_vegan","long_vegan"]], radius = 14).add_to(vegan_group)
vegan_group.add_to(map_1)
folium.LayerControl(collapsed = False).add_to(map_1)
map_1